In [11]:
from faker import Faker
import random
import bcrypt

fake = Faker()

NUM_USERS = 100
NUM_PARTICIPANTS = 1000
TASKS_PER_STUDY = 10
SUBTASKS_PER_TASK = 5
SESSIONS_PER_PARTICIPANT = 5

# Password3 because Idk how to hash diff passwords and be able to figure out what the unhashed password is to log in and test
example_hash = "$2a$10$wMENSNuvz4vNSzXM.0364e6SHxbElaU81gPVu/7/R.ZXSPDAQG.rm"
# When you actually make an acc you can choose the password and it will hash differently

user_ids = []
study_ids = []
participant_ids = []
task_ids = []

existing_titles = set()

study_to_tasks = {}

with open('data.sql', 'w') as f:

    for i in range(NUM_USERS):
        fname = fake.first_name().replace("'", "''")
        lname = fake.last_name().replace("'", "''")
        username = f"{fname.lower()}{i}"
        email = f"{username}@example.com"
        password = example_hash
        f.write(f"INSERT INTO Users(fname, lname, username, email, password) "
                f"VALUES ('{fname}', '{lname}', '{username}', '{email}', '{password}');\n")
        user_ids.append(i + 1)

    status_options = ['ACTIVE', 'COMPLETED', 'PLANNED']
    status_weights = [0.7, 0.2, 0.1]

    study_counter = 1

    for user_id in user_ids:
        num_studies = random.randint(1, 3)
        for _ in range(num_studies):
            # Ensure unique title
            title = fake.sentence(nb_words=4).replace("'", "''")
            while title in existing_titles:
                title = fake.sentence(nb_words=4).replace("'", "''")
            existing_titles.add(title)

            description = fake.text(max_nb_chars=100).replace("'", "''")
            platform = random.choice(['Web', 'Mobile', 'Desktop'])

            status = random.choices(status_options, weights=status_weights, k=1)[0]

            f.write(f"INSERT INTO Study(user_id, title, description, platform, status) "
                    f"VALUES ({user_id}, '{title}', '{description}', '{platform}', '{status}');\n")
            study_ids.append(study_counter)
            study_to_tasks[study_counter] = []
            study_counter += 1

    for i in range(NUM_PARTICIPANTS):
        fname = fake.first_name().replace("'", "''")
        lname = fake.last_name().replace("'", "''")
        age = random.randint(18, 65)
        occupation = fake.job().replace("'", "''")
        occupation_exp = f"{random.randint(1, 40)} years"
        email = f"{fname.lower()}.{lname.lower()}{i}@example.com"
        f.write(f"INSERT INTO Participant(fname, lname, age, occupation, occupation_exp, email) "
                f"VALUES ('{fname}', '{lname}', {age}, '{occupation}', '{occupation_exp}', '{email}');\n")
        participant_ids.append(i + 1)

    task_counter = 1
    for study_id in study_ids:
        for task_order in range(1, TASKS_PER_STUDY + 1):
            description = fake.sentence(nb_words=6).replace("'", "''")
            success_criteria = fake.sentence(nb_words=5).replace("'", "''")
            expected_time = random.randint(5, 60)
            f.write(f"INSERT INTO Task(study_id, task_order, description, success_criteria, expected_comp_time) "
                    f"VALUES ({study_id}, {task_order}, '{description}', '{success_criteria}', {expected_time});\n")
            task_ids.append(task_counter)
            study_to_tasks[study_id].append(task_counter)

            # Subtasks
            for subtask_id in range(1, SUBTASKS_PER_TASK + 1):
                sub_desc = fake.sentence(nb_words=4).replace("'", "''")
                sub_time = random.randint(1, 20)
                f.write(f"INSERT INTO Subtask(task_id, subtask_id, description, expected_comp_time) "
                        f"VALUES ({task_counter}, {subtask_id}, '{sub_desc}', {sub_time});\n")
            task_counter += 1

    session_counter = 1
    for participant_id in participant_ids:
        num_sessions = random.randint(1, SESSIONS_PER_PARTICIPANT)
        for _ in range(num_sessions):
            study_id = random.choice(study_ids)
            scheduled = fake.date_time_this_year()
            notes = fake.text(max_nb_chars=50).replace("'", "''")
            score = random.randint(0, 100)
            f.write(f"INSERT INTO Session(participant_id, study_id, scheduled, notes, score) "
                    f"VALUES ({participant_id}, {study_id}, '{scheduled}', '{notes}', {score});\n")

            tasks_in_study = study_to_tasks[study_id]
            for task_id in tasks_in_study:
                is_complete = random.choice([0, 1])
                completion_time = random.randint(5, 120)
                errors = random.randint(0, 5)
                result_notes = fake.text(max_nb_chars=50).replace("'", "''")
                f.write(f"INSERT INTO Result(session_id, task_id, is_complete, completion_time, errors, notes) "
                        f"VALUES ({session_counter}, {task_id}, {is_complete}, {completion_time}, {errors}, '{result_notes}');\n")

            session_counter += 1


In [12]:
with open('data.sql', 'r') as f:
    lines = f.readlines()
    print(''.join(lines[:20]))  # print first 20 lines

INSERT INTO Users(fname, lname, username, email, password) VALUES ('Lori', 'Pace', 'lori0', 'lori0@example.com', '$2a$10$wMENSNuvz4vNSzXM.0364e6SHxbElaU81gPVu/7/R.ZXSPDAQG.rm');
INSERT INTO Users(fname, lname, username, email, password) VALUES ('Jennifer', 'Tapia', 'jennifer1', 'jennifer1@example.com', '$2a$10$wMENSNuvz4vNSzXM.0364e6SHxbElaU81gPVu/7/R.ZXSPDAQG.rm');
INSERT INTO Users(fname, lname, username, email, password) VALUES ('Candace', 'Hoover', 'candace2', 'candace2@example.com', '$2a$10$wMENSNuvz4vNSzXM.0364e6SHxbElaU81gPVu/7/R.ZXSPDAQG.rm');
INSERT INTO Users(fname, lname, username, email, password) VALUES ('Sarah', 'Cardenas', 'sarah3', 'sarah3@example.com', '$2a$10$wMENSNuvz4vNSzXM.0364e6SHxbElaU81gPVu/7/R.ZXSPDAQG.rm');
INSERT INTO Users(fname, lname, username, email, password) VALUES ('Sherry', 'Lynch', 'sherry4', 'sherry4@example.com', '$2a$10$wMENSNuvz4vNSzXM.0364e6SHxbElaU81gPVu/7/R.ZXSPDAQG.rm');
INSERT INTO Users(fname, lname, username, email, password) VALUES ('Chri